# 🙤 **PARTIE Ⅰ** 🙧 
# Peuplement d'un graphe de connaissance sur l'œuvre photographique d'[Eugène Atget](https://fr.wikipedia.org/wiki/Eug%C3%A8ne_Atget) à partir des portails de données de la bibliothèque nationale de France

<div style="display: flex; justify-content: center; align-items: center; gap: 20px; flex-wrap: wrap;">
    <img src="assets/atget_1.jpg" alt="New Monico, Atget 1925" width="700" /> 
    <img src="assets/atget_2.jpg" alt="Portrait d'Eugène Atget vers 1890" width="282" />
</div>

🙑 Eugène Atget (1857‑1927) est un photographe français, pionnier de la photographie documentaire, actif au tournant du XX<sup>e</sup> siècle. Il a construit une œuvre photographique monumentale qui documente avec rigueur le Paris de la Belle Époque, de la Zone et ses environs, avant et pendant les profondes transformations urbaines du début du XXᵉ siècle. Ses images décrivent les rues, les façades, les boutiques, les cours, les jardins et les scènes de la vie quotidienne avec une précision visuelle remarquable, cherchant moins à créer une esthétique artificielle qu’à enregistrer minutieusement la réalité urbaine. Atget n’aspirait pas à être vu comme un artiste au sens traditionnel : il envisageait la photographie d’abord comme un outil documentaire permettant de fixer des éléments de la ville menacés de disparition. Cependant, au fil du temps Atget évolue de la simple documentation architecturale (1890-1905) et adopte un regard plus attentif aux traces de vie humaine et sociale, photographiant artisans, travailleurs des rues et habitants de la zone. L’accumulation de ces vues confère à son œuvre une puissance métaphysique et poétique qui a séduit bien au‑delà de son intention initiale, influençant notamment les surréalistes et des générations de photographes modernes. Dans sa diversité, l’œuvre d’Atget se situe à l’intersection du document social et historique et d’une forme naissante de lecture critique de l’espace urbain, interrogeant la visibilité des corps et des constructions dans le paysage citadin.
Aujourd’hui, la BnF conserve plus de 4 500 épreuves positives d’Atget, réparties en séries thématiques élaborées par lui‑même ou par la bibliothèque, ainsi que plusieurs albums qu’il a constitués. Cette immense archive, complétée par d’autres fonds publics et privés, fait d’Atget une figure centrale de la photographie documentaire et un témoignage unique du vieux Paris, oscillant entre la constatation archivistique et l’évocation sensible du temps qui passe. 🙗

<small>Source : Rizov, V. (2021). Eugène Atget and documentary photography of the city. Theory, Culture & Society, 38(3), 141-163. Pour en apprendre plus sur Eugène Atget et son oeuvre : https://expositions.bnf.fr/atget/arret/01.htm et https://gallica.bnf.fr/accueil/fr/html/sur-les-traces-du-paris-datget

Photos : à gauche le "New Monico", 1925. À droite : portrait photographique d'Eugène Atget vers 1890, auteur inconnu.
</small>

## 🙤 Objectifs

Cette première partie guide la construction d'une base de données locale indexant les photographies d'Eugène Atget archivées, inventoriées et numérisées par la BnF. Les métadonnées utiles sont exposées sur différents points d'accès de la BnF : [Gallica]() pour les numérisations, [le catalogue général de la bibliothèque]() pour les notices bibliographiques, [data.bnf.fr]() pour les métadonnées publiques en RDF.

Nous allons construire cette base de données sous la forme d'un **graphe de connaissances RDF** en collectant les informations sur les photographies de Atget à partir des **APIs** de la BnF. Ces donnes nous serviront plus tard pour **géolocaliser** réaliser un **explorateur cartographique interactif** du fond Atget de la BnF.

Le modèle du graphe de connaissances est pertinent  pour trois raisons : 
1.  les données sont **déjà partiellement disponibles en triplets RDF** ;
2.  les **métadonnées très imbriquées** de la BnF s'adaptent très bien à une représentation en graphe, mais très mal à de "simples" tableaux  ;
3.  un graphe de connaissance est **facile à enrichir** de nouvelles informations

Nous allons voir :
- les différents points d'accès aux métadonnées de la BnF et les modes de récupération liés : API SRU de Gallica, SPARQL endpoint de data.bnf.fr
- comment exécuter des requêtes SPARQL distantes en Python avec *SPARQLWrapper* pour récupérer des données ;
- comment manipuler un graphe RDF en Python avec *rdflib* ;
- comment visualiser un graphe de connaissances RDF en Python avec *PyVis*.

Légende des pictogrammes utilisés dans ce *notebook* : 
|Picto.| Légende|
| ---- | ---- |
|🎬 | Action à réaliser : à vous de jouer ! | 
|💡| Suggestion d'action complémentaire |
|🏗️/▶️| Cellule à compléter / exécuter|
|⚠️| Avertissement |
|ℹ️| Information supplémentaire ou astuce |
|📚| Ressources : documentation, article, etc.|

<hr/>

## 🙤 Chapitre 2 : introduction à SPARQLWrapper et RDFlib

Dans le chapitre 1, nous avons découvert le graphe de connaissances de la BnF et son modèle de données. Mais comment l'exploiter de manière plus automatisée ?

### Devenir une machine

Le point d'accès SPARQL https://data.bnf.fr/sparql/ est muni d'une interface graphique pour faire plaisir aux humains, mais son but premier est d'être utilisé de manière automatisée par des machines.

On peut exécuter une requête SPARQL en la passant dans le paramètre d'URL `query` d'une requête HTTP vers le point d'accès.

> 🎬 Exécuter par exemple la requête HTTP suivante dans un terminal avec cURL pour obtenir tous les triplets des entités **notice**, **manifestation** et **expression** de la photographie d'id ARK `cb402758340`
```sh
# Copiez et exécutez la commande dans un terminal 
# Au besoin  : sudo apt install curl
curl https://data.bnf.fr/sparql -X POST \
--data-urlencode 'query=
DESCRIBE 
  <http://data.bnf.fr/ark:/12148/cb402758340> 
  <http://data.bnf.fr/ark:/12148/cb402758340#about> 
  <http://data.bnf.fr/ark:/12148/cb402758340#Expression>' \
-H "Accept: text/turtle"
```

ℹ️ On passe le *header* `Accept: text/turtle` pour récupérer le résultat RDF avec la syntaxe Turtle, moins verbeuse que RDFXML et plus aisée à lire.

💡 Même au format Turle, le RDF n'est pas très lisible. Mais comme cela décrit un graphe, on peut le représenter visuellement ! Essayez par exemple de coller le code Turtle récupéré avec la commande cURL dans un visualisateur RDF comme  https://issemantic.net/rdf-visualizer.

### Requêtes SPARQL en Python avec SPARQLWrapper

Il est bien sûr possible d'exécuter des requêtes HTTP avec Python. On pourrait utiliser la bibliothèque généraliste `requests` pour cela, mais il existe une bibliothèque spécialisée nommée **SPARQLWrapper**.
Son nom est assez transparent : elle permet de créer un objet Python qui représente un point d'accès SPARQL distant que l'on peut ensuite interroger.

> ▶️ Installez SPARQLWrapper avec `uv` :

In [ ]:
!uv add SPARQLWrapper

📚 **RTFM** : la documentation de SQLWrapper : https://sparqlwrapper.readthedocs.io

> ▶️ Il suffit ensuite de créer un objet `SQLWrapper` qui pointe vers le point d'accès SPARQL de la BnF :

In [ ]:
from SPARQLWrapper import SPARQLWrapper

endpoint = SPARQLWrapper("https://data.bnf.fr/sparql") # ⚠️ Sans / final ! Pourquoi ? "https://data.bnf.fr/sparql" est l'adresse du endpoint SPARQL. "https://data.bnf.fr/sparql/" est celle de son interface graphique. 

> ▶️ On peut définir le format dans lequel on souhaite récupérer les résultats de requêtes. Par exemple pour obtenir des triplets en Turle :

In [ ]:
from SPARQLWrapper import TURTLE

endpoint.setReturnFormat(TURTLE)

📚 Les formats disponibles sont décrits dans la documentation : https://sparqlwrapper.readthedocs.io/en/latest/main.html#return-formats

> ▶️ On peut ensuite exécuter une requête SPARQL avec la méthode `setQuery` et récupérer directement le résultat au format souhaité avec les méthodes `setQuery` et `queryAndConvert`

In [ ]:
query = """DESCRIBE
                    <http://data.bnf.fr/ark:/12148/cb402758340>
                    <http://data.bnf.fr/ark:/12148/cb402758340#about>
                    <http://data.bnf.fr/ark:/12148/cb402758340#Expression>"""

endpoint.setQuery(query)

result: bytes = endpoint.queryAndConvert() # Le résultat dépend du format demandé plus haut, ici du texte Turtle. Ce texte est encodé en bytes pour être transporté sur le réseau ...
print(result.decode("utf-8")) # ... il faut donc le décoder en caractères UTF-8 pour l'afficher correctement.

### Créer un graphe RDF avec RDFlib

SPARQLWrapper permet très simplement d'interroger un point d'accès SPARQL et de récupérer des triplets dans le format souhaité. Cependant ce que l'on veut nous, c'est créer **notre propre graphe local** extrait de celui de la BnF et le manipuler en tant que tel.

C'est ici qu'intervient **RDFlib**, une bibliothèque Python qui permet exactement cela : créer ses propres graphes RDF ... comme des objets graphes !

> ▶️ On installe RDFlib avec `uv` :

In [ ]:
!uv add rdflib

> ▶️ Il est ensuite extrêmement facile de transformer en graphe les triplets récupérés précédemment :

In [ ]:
from rdflib import Graph

# 1. On initialise un graphe vide...
g = Graph()

# 2. ... puis on le remplit avec les données récupérées via SPARQLWrapper
g.parse(data=result, format="turtle")

# 3. On peut le sérialiser en Turtle vérifier qu'il contient bien les mêmes données
print(g.serialize(format="turtle"))

En fait, c'est même tellement facile qu'il est même possible de récupérer **directement un graphe rdflib** en sortie de `endpoint.queryAndConvert()` si l'on utilise les format `JSONLD` ou `XML`.

> 🏗️▶️ Complétez et exécutez la cellule suivante pour interroger le point d'accès SPARQL de data.bnf.fr et obtenir directement un objet `Graph` de rdflib.

In [ ]:
from SPARQLWrapper import JSONLD

endpoint.setQuery(query)

... # 🏗️ à compléter

g: Graph = ...

assert isinstance(g, Graph), "Le résultat n'est pas un graphe RDFlib !"

Évidemment on a plutôt envie de visualiser le graphe de manière un peu plus ... graphique. Malheureusement il n'existe pas de solution "clé en main" pour cela, mais par chance RDFlib est assez compatible avec la bibliothèque de visualisation de réseaux [PyVis](https://pyvis.readthedocs.io).

> ▶️ Le module utilitaire `utils.py` définit la fonction `show_graph()` qui accepte un objet `Graph` et l'affiche sous forme de réseau.

In [ ]:
!uv add pyvis # installer la bibliothèque pyvis pour la visualisation

from utils import show_graph

show_graph(g)

###  🏁 Fin du chapitre 2

Nous avons appris à interroger de manière très simple un point d'accès SPARQL et à représenter le résultat sous la forme d'un objet graphe manipulable graphe aux bibliothèques Python SPARQLWrapper et RDFlib.

Il est temps de passer aux choses sérieuses avec le chapitre 3 : extraire le graphe de connaissance des photographies d'Eugène Atget !